In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
from pathlib import Path

In [ ]:
CURRENT_DIR = Path.cwd()

In [ ]:
filepath = os.path.join(CURRENT_DIR, "S03_residual_stress_merge.xlsx")
dfm = pd.read_excel(filepath).rename(
    columns={"Sigma(x)": "sigma_x_pre", "FWHM": "FWHM_pre"}
)
dfm
dfm_imputed = dfm.copy()


In [ ]:
default_cols = ["section", "sample_no", "location"]

In [ ]:
dfm

# Prototype


In [ ]:
section = "AA5052"
y = "sigma_x_pre"
max_val = 150
min_val = -40
impute_idx_excels = [319]

# Filter data for specific section
filt = dfm["section"].isin([section])
idx_excel_col = "idx_excel_pre" if "pre" in y else "idx_excel_post"

# Identify rows with high residual stress in pre-weld data for further investigation
filt_val = (dfm[y] > max_val) | (dfm[y] < min_val)
dfm_outlier = dfm[filt & filt_val][[*default_cols, y, idx_excel_col]].sort_values(by=y)
display(dfm_outlier)


# Impute outlier values with median of respective location and section
# First convert index_excel to index
inpute_idxs = dfm[dfm[idx_excel_col].isin(impute_idx_excels)].index.tolist()
for idx, idx_excel in zip(inpute_idxs, impute_idx_excels):
    location = dfm.loc[idx, "location"]
    section = dfm.loc[idx, "section"]
    filt_idx = (
        (dfm["location"] == location)
        & (dfm["section"] == section)
        & (~dfm.index.isin(inpute_idxs))
    )
    mean_val = dfm[filt_idx][y].mean()
    print(
        f"Imputing idx_excel {idx_excel} at location {location}, section {section} with mean value: {mean_val}"
    )
    dfm_imputed.loc[idx, y] = mean_val

# Plot boxenplot of pre-weld residual stress by location and section
fig, ax = plt.subplots(2, 1, figsize=(10, 10), sharey=True)
sns.boxenplot(dfm[filt], x="location", y=y, ax=ax[0])
ax[0].plot(dfm_outlier["location"] - 1, dfm_outlier[y], "ro", label="Outliers")
for i, row in dfm_outlier.iterrows():
    ax[0].text(
        row["location"] - 1 + 0.2,
        row[y],
        f"{row[idx_excel_col]}",
        color="red",
        fontsize=10,
        ha="center",
        va="bottom",
        clip_on=True,
    )
ax[0].set_title(f"y: {y}, section: {section}")
sns.boxenplot(dfm_imputed[filt], x="location", y=y, ax=ax[1])
ax[1].set_title(f"y: {y}, section: {section} (Imputed)")
plt.tight_layout()
plt.show()

In [ ]:
dfm[filt]

# Function


In [ ]:
def outlier(section, y, max_val, min_val, impute_idxs):
    # section = "AA5052"
    # y = "sigma_x_pre"
    # max_val = 150
    # min_val = -40
    # impute_idxs = [370]

    # Filter data for specific section
    filt = dfm["section"].isin([section])
    idx_excel_col = "idx_excel_pre" if "pre" in y else "idx_excel_post"

    # Identify rows with high residual stress in pre-weld data for further investigation
    filt_val = (dfm[y] > max_val) | (dfm[y] < min_val)
    dfm_outlier = dfm[filt & filt_val][[*default_cols, y, idx_excel_col]].sort_values(
        by=y
    )
    display(dfm_outlier)

    # Impute outlier values with median of respective location and section
    # First convert index_excel to index
    inpute_idxs = dfm[dfm[idx_excel_col].isin(impute_idx_excels)].index.tolist()
    for idx, idx_excel in zip(inpute_idxs, impute_idx_excels):
        location = dfm.loc[idx, "location"]
        section = dfm.loc[idx, "section"]
        filt_idx = (
            (dfm["location"] == location)
            & (dfm["section"] == section)
            & (~dfm.index.isin(inpute_idxs))
        )
        mean_val = dfm[filt_idx][y].mean()
        print(
            f"Imputing idx_excel {idx_excel} at location {location}, section {section} with mean value: {mean_val}"
        )
        dfm_imputed.loc[idx, y] = mean_val

    # Plot boxenplot of pre-weld residual stress by location and section
    fig, ax = plt.subplots(2, 1, figsize=(10, 10), sharey=True)
    sns.boxenplot(dfm[filt], x="location", y=y, ax=ax[0])
    ax[0].plot(dfm_outlier["location"] - 1, dfm_outlier[y], "ro", label="Outliers")
    ax[0].set_title(f"y: {y}, section: {section}")
    for i, row in dfm_outlier.iterrows():
        ax[0].text(
            row["location"] - 1 + 0.2,
            row[y],
            f"{row[idx_excel_col]}",
            color="red",
            fontsize=10,
            ha="center",
            va="bottom",
            clip_on=True,
        )
    sns.boxenplot(dfm_imputed[filt], x="location", y=y, ax=ax[1])
    ax[1].set_title(f"y: {y}, section: {section} (Imputed)")
    plt.tight_layout()
    plt.show()

# Outlier Removal and Imputation


In [ ]:
section = "AA5052"
y = "sigma_x_pre"
max_val = 150
min_val = -40
impute_idx_excels = [370]

outlier(section, y, max_val, min_val, impute_idx_excels)

In [ ]:
section = "AA6061"
y = "sigma_x_pre"
max_val = 150
min_val = -40
impute_idx_excels = []

outlier(section, y, max_val, min_val, impute_idx_excels)

In [ ]:
section = "AA5052"
y = "sigma_x_post"
max_val = 150
min_val = -40
impute_idx_excels = []

outlier(section, y, max_val, min_val, impute_idx_excels)

In [ ]:
section = "AA6061"
y = "sigma_x_post"
max_val = 150
min_val = -40
impute_idx_excels = []

outlier(section, y, max_val, min_val, impute_idx_excels)

In [ ]:
section = "Center"
y = "sigma_x_post"
max_val = 150
min_val = -40
impute_idx_excels = []

outlier(section, y, max_val, min_val, impute_idx_excels)

In [ ]:
section = "AA5052"
y = "FWHM_pre"
max_val = 150
min_val = -40
impute_idx_excels = []

outlier(section, y, max_val, min_val, impute_idx_excels)

In [ ]:
section = "AA6061"
y = "FWHM_pre"
max_val = 150
min_val = 1
impute_idx_excels = []

outlier(section, y, max_val, min_val, impute_idx_excels)

In [ ]:
section = "AA5052"
y = "FWHM_post"
max_val = 150
min_val = 1
impute_idx_excels = []

outlier(section, y, max_val, min_val, impute_idx_excels)

In [ ]:
section = "AA6061"
y = "FWHM_post"
max_val = 150
min_val = 1
impute_idx_excels = []

outlier(section, y, max_val, min_val, impute_idx_excels)

In [ ]:
section = "Center"
y = "FWHM_post"
max_val = 150
min_val = 1
impute_idx_excels = []

outlier(section, y, max_val, min_val, impute_idx_excels)

In [ ]:
dfm_imputed.to_excel("S04_residual_stress_imputed.xlsx", index=False)